In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv


In [2]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # or "gpt2-medium" for a larger model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
def generate_text(prompt, max_length=200):
    # Tokenize input text
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids, 
            max_length=max_length, 
            do_sample=True, 
            temperature=0.7
        )
    
    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "What are the symptoms of anxiety?"
generated_text = generate_text(prompt)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What are the symptoms of anxiety?

As you get used to the feeling, you get more and more anxious. Sometimes you will feel the tension. In other times you will feel that something is off. You can feel the heat coming from a drawer and you might even feel a buzz, but it can be very cold, it can be very hot, and you'll want to think about the idea of not doing anything because you're going to feel that.

In your head, your mind, your body, your body is at this moment that you should make this decision. You may feel like you're not going to do anything. You may feel like you're not going to do anything. You may feel like you're not going to do anything. You may feel like you're not going to do anything. It's very difficult to manage.

You may feel like you're not going to do anything. You may feel like you're not going to do anything. You


In [8]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load your dataset
df = pd.read_csv('/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv')

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

# Clean and combine text columns if necessary
df['Questions'] = df['Questions'].apply(lambda x: x.lower())
df['Answers'] = df['Answers'].apply(lambda x: x.lower())
df['text'] = df['Questions'] + " " + df['Answers']

# Generate a response for the first 10 rows
def generate_text(prompt, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Apply the text generation only for the first 10 rows
df.loc[:9, 'Generated_Response'] = df.loc[:9, 'text'].apply(lambda x: generate_text(x, max_length=300))

# Check if the 'Generated_Response' column exists and has data
if 'Generated_Response' in df.columns:
    print("Generated_Response column created successfully.")
    print(df.loc[:9, 'Generated_Response'])
else:
    print("Failed to create Generated_Response column.")


Generated_Response column created successfully.
0    what does it mean to have a mental illness? me...
1    who does mental illness affect? it is estimate...
2    what causes mental illness? it is estimated th...
3    what are some of the warning signs of mental i...
4    can people with mental illness recover? when h...
5    what should i do if i know someone who appears...
6    how can i find a mental health professional fo...
7    what treatment options are available? just as ...
8    if i become involved in treatment, what do i n...
9    what is the difference between mental health p...
Name: Generated_Response, dtype: object


In [9]:
for response in df.loc[:7, 'Generated_Response']:
    print(response)

what does it mean to have a mental illness? mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. they are associated with distress and diminished capacity to engage in the ordinary activities of daily life.
mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. on the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. these include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.
it is important to know that mental illnesses are medical conditions that have nothing to do with a personâ€™s character, intelligence, or willpower. just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brainâ€™s biology.
similarly to how one wo